In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import wave as w
import copy
np.set_printoptions(precision=2)
from cycler import cycler
plt.rcParams.update(plt.rcParamsDefault)

# Example usage of simulated materials
This notebook is ment as an reference on how to use `Spectrum` and `Material` classes,
and for visual test of the implementation.

### Generate an example spectrum

In [ ]:
s = w.GaussianSpectrum(mean=w.GREEN, std=(5*w.MICRO))
# s = w.DeltaSpectrum(wavelength=w.GREEN)
z = np.linspace(-3*w.MICRO, 3*w.MICRO, 1000)
s.delay(0)
s.plot(wavelength=True)
plt.show()
s.plot_amplitude(z)
plt.show()

### Compare different materials

#### Analitical vs constant

In [ ]:
air = w.EmptySpace(z=z)
air.plot()
plt.show()

In [ ]:
common_n = 1
fixed_dielectric = w.ConstantMaterial(z=z, n0=common_n)
fixed_dielectric.plot()
plt.show()

In [ ]:
dielectric = w.SimpleDielectric(z=z, n0=1, max_dn=0.2, min_energy=4e6, max_energy=1.2e7)
dielectric.plot()
plt.show()

In [ ]:
plt.plot(z, air.energy_distribution(s,s), label="analitical")
plt.plot(dielectric.z[:-1], dielectric.energy_distribution(s, s)[:-1], label="numerical - wave transfer matrices")
plt.plot(fixed_dielectric.z, fixed_dielectric.energy_distribution(s, s), label="numerical - phase shift")
plt.legend()
plt.show()

### An example recording

In [ ]:
dielectric.record(s, s)

In [ ]:
plt.plot(dielectric.z[:-1], dielectric.deposited_energy[:-1])
plt.title("deposited energy")
plt.show()

dielectric.plot(plot_imaginary=False)
plt.show()

In [ ]:
dielectric = w.SimpleDielectric(z=z, n0=1, max_dn=0.2, min_energy=4e6, max_energy=1.2e7)
plt.plot(dielectric.z[:-1], dielectric.energy_distribution(s, s)[:-1], label="numerical - wave transfer matrices")
plt.plot(fixed_dielectric.z, fixed_dielectric.energy_distribution(s, s), label="numerical - phase shift")
# plt.plot(z, intf.intensity(z=z), label="analitical", alpha=0.1)
plt.legend()
plt.show()

In [ ]:
plt.rcParams['axes.prop_cycle'] = cycler('color', plt.get_cmap('viridis')(np.linspace(0, 1, 21)))

In [ ]:
dielectric = w.SimpleDielectric(z=z, n0=1, max_dn=0.2, max_visible_energy=4, min_energy=4e6, max_energy=1.5e7)
for pulse in range(20):
    phase = np.pi * pulse / 40
    s2 = s * np.exp(1j*phase)
    dielectric.record(s, s2)
    plt.plot(dielectric.z[:-1], dielectric.recent_energy[:-1], label="pulse {}".format(pulse + 1), alpha=0.5)
    
plt.legend(loc=3)
plt.show()

In [ ]:
dielectric = w.SimpleDielectric(z=z, n0=1, max_dn=0.2, max_visible_energy=5, min_energy=4e6, max_energy=1.5e7)
for pulse in range(20):
    phase = np.pi * pulse / 40
    s2 = s * np.exp(1j*phase)
    dielectric.plot(plot_imaginary=False, alpha=0.5)
    dielectric.record(s, s2)
    
plt.legend()
plt.show()